In [3]:
import sys
import importlib
import nltk

nltk.download('punkt') # Example download

# Add the shared folder to path
sys.path.append('../shared')

# Force reload the module to get the latest changes
if 'NLPMethods' in sys.modules:
    importlib.reload(sys.modules['NLPMethods'])

from NLPMethods import NLPMethods

# Create an instance of the NLPMethods class
nlp_methods = NLPMethods()

# Call the hello_from_class method
nlp_methods.hello_from_class()

# Fetch the corpus and store the results
corpus_data = nlp_methods.fetch_corpus("https://www.gutenberg.org/cache/epub/2500/pg2500.txt")

print(corpus_data['quote_sentences'])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marobinette/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


hello from class
Number of sentences: 2080
Number of quotes found: 383
Quote sentences: 104
Non-quote sentences: 1976
Quote tokens: 0
Non-quote tokens: 42935
['“Your soul is the whole world”, was written there, and it was written that man in his sleep, in his deep sleep, would meet with his innermost part and would reside in the Atman', '“Govinda,” Siddhartha spoke to his friend, “Govinda, my dear, come with me under the Banyan tree, let’s practise meditation', '“O Siddhartha,” he exclaimed, “will your father permit you to do that', '“O Govinda,” he spoke quietly, “let’s not waste words', '“Siddhartha,” he spoke, “what are you waiting for', '“You will,” he spoke, “go into the forest and be a Samana', '“You have come,” said Siddhartha and smiled', '“I have come,” said Govinda', '“How do you think, Govinda,” Siddhartha spoke one day while begging this way, “how do you think did we progress', '” “If you only,” spoke Govinda, “wouldn’t speak such terrible words, Siddhartha', '“Oh Siddharth